In [1]:

import os

import tensorflow as tf
tf.device('/gpu:1')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1
session = tf.compat.v1.Session(config=config)

import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import PreProcessing
import net
import PatchExtractor

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Physical devices cannot be modified after being initialized


RuntimeError: Could not find an available GPU after 1 attempts with 900 seconds interval.

# Variables

In [ ]:
in_path = 'giuriati_2/20170621_deg0_HHVV.npy'
out_path = 'cnn_article'
architecture = 'Auto3D2'
ny = 3 # number of adjacent B-scans to be considered
data_augmentation = True
preprocessing = 'normalize'
patch_size = 64  ##64
patch_stride = 4 ##4
n_bsc = 5 #5 number of B_scans for training

In [ ]:
def parse_pp(string):
    return getattr(PreProcessing, string)

def parse_net(string):
    return getattr(net, string)

In [ ]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

field, campaign = in_path.split('/')
campaign, extension = campaign.split('.')

# Load Datasets

In [ ]:
dataset = np.load('./datasets/'+str(in_path),allow_pickle=True).item()

train_bsc_idx = np.where(np.asarray(dataset['ground_truth']) == 0)[0][:n_bsc]
trainset = dataset['data'][train_bsc_idx]
trainset = np.moveaxis(trainset, np.argmin(trainset.shape), -1)
del dataset

### block extraction

In [ ]:
if patch_size is not None:
    patch_size = (patch_size, patch_size)
else:
    patch_size = trainset.shape[1:]

patch_size = patch_size + (ny,)
patch_stride = (patch_stride, patch_stride, 1)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
from tqdm import tqdm

In [2]:
training = './cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3'
net_weights = training + '.h5'


### Load dataset

In [3]:
# in this case, the dataset for training in the same for testing
train_path = in_path
dataset = np.load('./datasets/' + str(in_path),allow_pickle=True).item()

# preprocessing
data = dataset['data']

# patch extractor
pe = PatchExtractor.PatchExtractor(patch_size, stride=patch_stride)

# background bscans for training
gt = np.asarray(dataset['ground_truth'])
del dataset

test_idx = np.arange(data.shape[0])
# check whether the test dataset is the same of the training
if in_path == train_path:
    train_idx = np.where(gt == 0)[0][:n_bsc]
    test_idx = np.delete(test_idx, train_idx)
testset = data[test_idx]
gt = gt[test_idx]
del data
testset = np.moveaxis(testset, np.argmin(testset.shape), -1)


NameError: name 'in_path' is not defined

In [4]:
testset.shape
gt.shape
gt=gt[:7]
testset=testset[:,:,:7]

NameError: name 'testset' is not defined

In [11]:
data.shape[0]

NameError: name 'data' is not defined

### Load architecture

In [12]:
autoencoder, encoder = parse_net(architecture)(patch_size)
autoencoder.load_weights(os.path.join(net_weights))

out_name = field+'_'+campaign+'_'+architecture+'_patch'+str(patch_size[0])+'_stride'+str(patch_stride[0])+'_bsc'+str(n_bsc)+'_ny'+str(ny)

In [13]:
encoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_in_0 (InputLayer)          [(None, 64, 64, 3)]       0         
_________________________________________________________________
g_conv_0 (Conv2D)            (None, 64, 64, 16)        1744      
_________________________________________________________________
g_conv_1 (Conv2D)            (None, 32, 32, 16)        6416      
_________________________________________________________________
g_conv_2 (Conv2D)            (None, 16, 16, 16)        4112      
_________________________________________________________________
g_conv_3 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 4, 16)          1040      
_________________________________________________________________
encoder (Conv2D)             (None, 2, 2, 16)          272 

In [14]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
g_in_0 (InputLayer)          [(None, 64, 64, 3)]       0         
_________________________________________________________________
g_conv_0 (Conv2D)            (None, 64, 64, 16)        1744      
_________________________________________________________________
g_conv_1 (Conv2D)            (None, 32, 32, 16)        6416      
_________________________________________________________________
g_conv_2 (Conv2D)            (None, 16, 16, 16)        4112      
_________________________________________________________________
g_conv_3 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 4, 16)          1040      
_________________________________________________________________
encoder (Conv2D)             (None, 2, 2, 16)          272   

### Test loop

In [15]:

patches = pe.extract(testset)
del testset
print(patches.shape)

(27, 95, 5, 64, 64, 3)


In [16]:
patchesIdx = patches.shape
print("check00")
patches_hat = autoencoder.predict(patches.reshape((-1,) + patch_size))
print("check01")
mseFeat = (encoder.predict(patches.reshape((-1,) + patch_size)) - encoder.predict(patches_hat))**2
del patches
uv10001=patches_hat.shape
del patches_hat
print("check 1")
mseFeat_patches = np.zeros(uv10001) + np.mean(mseFeat, axis=(1,2,3)).reshape((-1,1,1,1))

del mseFeat
mseFeat_vol = pe.reconstruct(mseFeat_patches.reshape(patchesIdx))
print(mseFeat_patches)
del mseFeat_patches

check00
check01
check 1
[[[[0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   ...
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]]

  [[0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   ...
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]]

  [[0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   ...
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]]

  ...

  [[0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   ...
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]
   [0.00043441 0.00043441 0.00043441]]

  [[0.00043441 0.0004

### Evaluation

In [17]:
mse_mask_max = np.max(mseFeat_vol, axis=(0, 1))
fpr_max, tpr_max, thresholds_max = roc_curve(gt, mse_mask_max)
roc_auc_max = roc_auc_score(gt, mse_mask_max)
print('best AUC = %0.2f' % roc_auc_max)

best AUC = 0.00


In [22]:
train_path = in_path
dataset = np.load('./datasets/' + str(in_path),allow_pickle=True).item()

# preprocessing
data = dataset['data']

In [24]:
data.shape

(66, 170, 440)

In [25]:
np.arange(data.shape[0])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65])

In [28]:
print(mse_mask_max)

[0.00051133 0.00049488 0.0004966  0.00050032 0.00048536 0.00047857
 0.0004706 ]


In [30]:
mseFeat_vol

array([[[0.00043441, 0.00042101, 0.00040573, ..., 0.00038516,
         0.00039015, 0.0003808 ],
        [0.00043441, 0.00042101, 0.00040573, ..., 0.00038516,
         0.00039015, 0.0003808 ],
        [0.00043441, 0.00042101, 0.00040573, ..., 0.00038516,
         0.00039015, 0.0003808 ],
        ...,
        [0.00034817, 0.00038492, 0.00039232, ..., 0.00039415,
         0.00038767, 0.00037558],
        [0.00034817, 0.00038492, 0.00039232, ..., 0.00039415,
         0.00038767, 0.00037558],
        [0.00034817, 0.00038492, 0.00039232, ..., 0.00039415,
         0.00038767, 0.00037558]],

       [[0.00043441, 0.00042101, 0.00040573, ..., 0.00038516,
         0.00039015, 0.0003808 ],
        [0.00043441, 0.00042101, 0.00040573, ..., 0.00038516,
         0.00039015, 0.0003808 ],
        [0.00043441, 0.00042101, 0.00040573, ..., 0.00038516,
         0.00039015, 0.0003808 ],
        ...,
        [0.00034817, 0.00038492, 0.00039232, ..., 0.00039415,
         0.00038767, 0.00037558],
        [0.0